# *CROP PREDICTION USING SOIL PARAMETERS AND REAL TIME WEATHER CONDITIONS*

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly as py
import plotly.graph_objs as go
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
import warnings
import os
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold

In [ ]:
#Load File
df = pd.read_csv('/kaggle/input/final-crop-prediction-data/Crop_recommendation (1).csv')
df.head(5)

In [ ]:
df = df.rename(columns={'N': 'Nitrogen (N)'})
df = df.rename(columns={'P': 'Phosphorous (P)'})
df = df.rename(columns={'K': 'Potassium (K)'})

In [ ]:
#Exploratory Data Analysis

In [ ]:
df.dtypes
#All data types look good, no need to change any data types.

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()
#None null values, all look good so far.

In [ ]:
#Check if data is balanced
bplot = sns.countplot(y='label',data=df, palette="muted")
bplot.set_ylabel('Crop', fontsize=10)
bplot.set_xlabel('Count', fontsize=10) 
bplot.tick_params(labelsize=11)

In [ ]:
#data set without column 'label'
df_2 = df.drop(columns=['label'])

In [ ]:
for i in range(0, 6, 3):
   pplot = sns.pairplot(data=df,
                x_vars=df.columns[i:i+4],
                y_vars=['label'], height=3)
   pplot.tick_params(labelsize=9)  

In [ ]:
sns.pairplot(data=df, hue = 'label')

In [ ]:
# Create a box plot for my data  
splot = sns.boxplot(data=df_2, whis=np.inf,                              
        width=0.5, linewidth = 0.7)

# Labels and clean up on the plot                                                                                                                                                                                                                                                                                              
splot.set_ylabel('Values', fontsize=10)
splot.set_xlabel('Features', fontsize=10)  
splot.tick_params(labelsize=8)                                                                    
splot.yaxis.grid(True, clip_on=False)   
plt.xticks(rotation=45)
splot.grid(False)

In [ ]:
cormat = df_2.corr()
round(cormat,2)
hplot = sns.heatmap(cormat,cmap="Greens", annot=True, annot_kws={"size": 8})
hplot.tick_params(labelsize=9)
plt.xticks(rotation=45)

In [ ]:
#Data Pre-Processing

In [ ]:
#Convert labels into categories codes
r = df.label.astype('category')
response = dict(enumerate(r.cat.categories))
df['response']=r.cat.codes

#Declare and set y and X variables
y=df.response
X=df[['Nitrogen (N)','Phosphorous (P)','Potassium (K)','temperature','humidity','ph','rainfall']]

In [ ]:
#Data Split in training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=10)

In [ ]:
len(X_train)

In [ ]:
#Feature Scaling
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model_svm = svm.SVC(kernel='rbf', C=10)

In [ ]:
#Training of classifier on training data
model_svm.fit(X_train_scaled, y_train)

#Prediction
y_pred = model_svm.predict(X_test_scaled)

#Model Accuracy
accuracy = round(accuracy_score(y_test, y_pred),3)
print("Accuracy: ", accuracy)

In [ ]:
#Precision, recall, and F1 score
precision = round(precision_score(y_test, y_pred, average='weighted',zero_division=1),3)
recall = round(recall_score(y_test, y_pred, average='weighted'),3)
f1 = round(f1_score(y_test, y_pred, average='weighted'),3)

# Print the precision, recall, and F1 score
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1) 

In [ ]:
# K-fold Cross Validation

kfold = KFold(n_splits=5, shuffle=True, random_state=10)

#k-fold iteration
scores = []
for train_index, test_index in kfold.split(X):
    X_train3, y_train3 = X.iloc[train_index], y.iloc[train_index]
    X_test3, y_test3 = X.iloc[test_index], y.iloc[test_index]
    model_svm.fit(X_train3, y_train3)

    y_pred = model_svm.predict(X_test3)
    score = accuracy_score(y_test3, y_pred)

    #Append the accuracy score for this fold to the list of scores
    scores.append(score)

#Mean accuracy across all folds
mean_accuracy = round(sum(scores) / len(scores),3)

print("Mean Accuracy:", mean_accuracy)

In [ ]:
#-----------------------------------------Implementation of Decision Tree Model-----------------------------------------------

In [ ]:
#Decision Tree classifier object
model_dt = DecisionTreeClassifier(max_depth = 10, criterion = 'gini', random_state=10)

In [ ]:
#Training of classifier on training data
model_dt.fit(X_train_scaled, y_train)

In [ ]:
#Prediction
y_pred = model_dt.predict(X_test_scaled)

#Model Accuracy
accuracy = round(accuracy_score(y_test, y_pred),3)
print("Accuracy: ", accuracy)

In [ ]:
#Precision, recall, and F1 score
precision = round(precision_score(y_test, y_pred, average='weighted',zero_division=1),3)
recall = round(recall_score(y_test, y_pred, average='weighted'),3)
f1 = round(f1_score(y_test, y_pred, average='weighted'),3)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
# K-fold Cross Validation

kfold = KFold(n_splits=5, shuffle=True, random_state=10)

#k-fold iteration
scores = []
for train_index, test_index in kfold.split(X):
    X_train2, y_train2 = X.iloc[train_index], y.iloc[train_index]
    X_test2, y_test2 = X.iloc[test_index], y.iloc[test_index]
    
    model_dt.fit(X_train2, y_train2)

    y_pred = model_dt.predict(X_test2)
    score = accuracy_score(y_test2, y_pred)

    #Append the accuracy score for this fold to the list of scores
    scores.append(score)

#Mean accuracy across all folds
mean_accuracy = round(sum(scores) / len(scores),3)

print("Mean accuracy:", mean_accuracy)

In [ ]:
#-----------------------------------------Implementation of Random Forest-----------------------------------------------

In [ ]:
model_rf = RandomForestClassifier(max_depth=6, n_estimators=120, random_state=10)

In [ ]:
model_rf.fit(X_train_scaled, y_train)

# * _*********************************_OUTPUTS_*********************************_ *

In [ ]:
#Prediction
y_pred = model_rf.predict(X_test_scaled)

#Model Accuracy
accuracy = round(accuracy_score(y_test, y_pred),3)
print("Accuracy: ", accuracy)

In [ ]:
## print("Please enter data for prediction:")
data_input = {}
for column in X.columns:
    data_input[column] = float(input(f"{column}: "))

# Create a DataFrame from the input data
input_data = pd.DataFrame([data_input])

# Preprocess the input data (scaling)
X_input = scaler.transform(input_data)

# Use the trained model to make predictions
predictions = model_rf.predict(X_input)

class_label_mapping = {
    0: 'Maze',
    1: 'Chickpeea',
    2: 'Kidney Beans',
    3: 'Pigeon Peas',
    4: 'Moth Beans',
    5: 'Mung Bean',
    6: 'Black Gram',
    7: 'Lentil',
    8: 'Pomegranate',
    9: 'Banana',
    10: 'Mango',
    11: 'Grapes',
    12: 'Water Melon',
    13: 'Musk Melon',
    14: 'Apple',
    15: 'Orange',
    16: 'Papaya',
    17: 'Coconut',
    18: 'Cotton',
    19: 'Jute',
    20: 'Tomato',
    21: 'Coffee',

    # Add more mappings as needed
}

# Display the predictions
# print("Predicted Class:", predictions[0])443
predicted_class = class_label_mapping.get(predictions[0], 'Unknown')

# Display the predicted class and its actual value
print("\n your Estimated crop is :", predicted_class)

In [ ]:
#Precision, recall, and F1 score
precision = round(precision_score(y_test, y_pred, average='weighted',zero_division=1),3)
recall = round(recall_score(y_test, y_pred, average='weighted'),3)
f1 = round(f1_score(y_test, y_pred, average='weighted'),3)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
# K-fold Cross Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=10)

#k-fold iteration
scores = []
for train_index, test_index in kfold.split(X):
    X_train_k, y_train_k = X.iloc[train_index], y.iloc[train_index]
    X_test_k, y_test_k = X.iloc[test_index], y.iloc[test_index]
    model_rf.fit(X_train_k, y_train_k)

    y_pred = model_rf.predict(X_test_k)
    score = accuracy_score(y_test_k, y_pred)

    #Append the accuracy score for this fold to the list of scores
    scores.append(score)

#Mean accuracy across all folds
mean_accuracy = round(sum(scores) / len(scores),3)

print("Mean Accuracy:", mean_accuracy)

In [ ]:
plt.figure(figsize=(5, 3), dpi=80)
importance_sorted = sorted(zip(model_rf.feature_importances_, X_train.columns), reverse=True)
feature_importance = [imp for imp, _ in importance_sorted]
feature_names = [name for _, name in importance_sorted]

c_features = len(feature_importance)
plt.bar(range(c_features), feature_importance, color = "green")
plt.ylabel("Feature Importance")
plt.xlabel("Feature Name")
plt.xticks(np.arange(c_features), feature_names, rotation=45)

# # **CROP MONTINORING SYSTEM USING IMAGE**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.applications import VGG16
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_ds="/kaggle/input/tomato/train"
test_ds="/kaggle/input/tomato/valid"

In [ ]:
os.listdir('/kaggle/input/tomato/train')

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)#initialize train generator 
                                 


valid_datagen = ImageDataGenerator(rescale = 1.0/255.) #initialize validation generator 

test_datagen = ImageDataGenerator(rescale = 1.0/255.) #initialize test generator

In [ ]:
train_datagen = ImageDataGenerator(zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15)

test_datagen = ImageDataGenerator()


train_generator = train_datagen.flow_from_directory(train_ds,target_size=(224, 224),batch_size=32,shuffle=True,class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_ds,target_size=(224,224),batch_size=32,shuffle=False,class_mode='categorical')


In [ ]:
train_generator[0][0].shape

In [ ]:
img= train_generator[0]
print(img)

In [ ]:
type(train_generator)

In [ ]:
img = train_generator[0]
print(img[0].shape) # shape of the input batch
print(img[1].shape) # shape of the target labels

In [ ]:
print(len(train_generator))
print(len(test_ds))

In [ ]:
import matplotlib.pyplot as plt
from skimage import io

# Load image
img_url = "/kaggle/input/tomato/train/Late_blight/014eee0c-8000-4762-b4b8-63ecc2fadc13___GHLB2ES Leaf 69.1.JPG"
img = io.imread(img_url)

# Display image
plt.imshow(img)
plt.axis('off')
plt.show()